In [1]:
import spacy
import re

In [2]:
from spacy.language import Language
from spacy.tokens import Doc
from spacy.matcher import PhraseMatcher

In [5]:
# !pip install pyjsgf

In [4]:
import jsgf
from jsgf import PublicRule,Literal,Rule
from jsgf import Grammar,grammars
from jsgf import parser,parse_rule_string,expansions,parse_expansion_string

In [6]:
rule1=PublicRule(name='music_play',expansion='[can you](put on| play)(<artist>|<song>)')
rule2=Rule(name='artist',visible=True,expansion='beatles |radio head |cake |pink floyd')
rule3=Rule(name='song',visible=True,expansion='confortably numb |paranoid android |let it be |hey jude')

rule1_compiled=rule1.compile(ignore_tags=True)
print('rule_compiled',rule1_compiled)
rule1_expanded=rule1.expansion
print('rule1_expansion',rule1_expanded)
rule1_sring_expand=rule1_expanded.generate()
print('rule1_string_generate',rule1_sring_expand)

print('-------------------------------------------')

rule2_compiled=rule2.compile()
print('rule2_compiled',rule2_compiled)
rule2_expanded=rule2.expansion
print('rule2_expansion',rule2_expanded)
rule2_string_expand=rule2_expanded.generate()
print('rule2_string',rule2_string_expand)


rule_compiled public <music_play> = [can you](put on| play)(<artist>|<song>);
rule1_expansion Literal('[can you](put on| play)(<artist>|<song>)')
rule1_string_generate [can you](put on| play)(<artist>|<song>)
-------------------------------------------
rule2_compiled public <artist> = beatles |radio head |cake |pink floyd;
rule2_expansion Literal('beatles |radio head |cake |pink floyd')
rule2_string beatles |radio head |cake |pink floyd


In [7]:
# itertools.product()

In [8]:
grammar=Grammar('music_play')
grammar.add_rules(rule1,rule2,rule3)

compiled_grammar=grammar.compile()
print('compiled_grammar\n',compiled_grammar)

parsed_grammer=parser.parse_grammar_string(compiled_grammar)
print('parsed_grammar\n',parsed_grammer)

compiled_grammar
 #JSGF V1.0;
grammar music_play;
public <music_play> = [can you](put on| play)(<artist>|<song>);
public <artist> = beatles |radio head |cake |pink floyd;
public <song> = confortably numb |paranoid android |let it be |hey jude;

parsed_grammar
 Grammar(version=1.0, charset=<auto>, language=<auto>, name=music_play)


In [36]:
for rule in parsed_grammer.rules:
    seq=rule.expansion
    print(seq)

    break 

Sequence(OptionalGrouping(Literal('can you')), ParsedAlternativeSet(Literal('put on'), Literal('play')), ParsedAlternativeSet(NamedRuleRef('artist'), NamedRuleRef('song')))


In [41]:
for rule in grammar.rules:
    print(rule,'\n',rule.expansion)
    
    break
    
#     print(expansions.Expansion.generate())


PublicRule(name='music_play', expansion=Literal('[can you](put on| play)(<artist>|<song>)')) 
 Literal('[can you](put on| play)(<artist>|<song>)')


In [47]:
# expansions.Literal(rule.expansion)

In [52]:
rule1.__reduce__()

<function PublicRule.__reduce__()>

### Grammar for handling Next level of Handling

public <music_play> =
[i want to listen to] (<genre> | <song> | <album> |<artist>) [music]
[play me] (<song> | <album>) [by] (<artist>);

<artist> =

beatles |

radio head |

cake |

pink floyd;

<song> =

comfortably numb |

paranoid android |

let it be |

hey jude;

<genre> =

jazz;

<album> =ummagumma;


#### Generated Utterances
File is too big so is stored in text file as Recursive Utterances

## To Handle unknown words

In [ ]:

def _compile_literal(self, element, *args, **kwargs):
    # Build literals as sequences and use <NULL> for unknown words.
    children = []
    for word in element.words:
        if self.engine.check_valid_word(word):
            children.append(jsgf.Literal(word))
        else:
            children.append(self.compile_element(elements_.Impossible(), *args, **kwargs))

            # Save the unknown word.
            args[1].add(word)

    return jsgf.Sequence(*children)

### Examples to know workflows
#### git repo link
#### https://github.com/Danesprite/pyjsgf/tree/master/examples

In [87]:

from jsgf import PublicRule, Grammar, AlternativeSet, Literal, Sequence


def main():
    # Define a open/close file rule.
    open, close = Literal("open"), Literal("close")
    open.tag, close.tag = "OPEN", "CLOSE"
    cmd = PublicRule("command", Sequence(AlternativeSet(open, close), "the file"))
    print(cmd)
    # Print the tags of the 'command' rule.
    print("Tags: %s\n" % cmd.tags)

    # Initialise a new grammar and add the rule to it.
    g = Grammar()
    g.add_rule(cmd)

    # Print the compiled grammar
    print("Compiled grammar is:\n%s" % g.compile())

    # Find and print rules tagged with "OPEN"
    print("Tagged rules are:\n%s\n" % g.find_tagged_rules("OPEN"))
    print("Tagged rules are:\n%s\n" % g.find_tagged_rules("CLOSE"))
        
        
    # Matching tags can be retrieved using r.get_tags_matching
    # The Rule.matched_tags property can also be used if Rule.matches or
    # Grammar.find_matching_rules has been called first.
    speech = "open the file"
    print("Tags matching '%s' are: %s" % (speech, cmd.get_tags_matching(speech)))
    print('generated::',cmd.generate())


if __name__ == '__main__':
    main()

PublicRule(name='command', expansion=Sequence(AlternativeSet(Literal('open'), Literal('close')), Literal('the file')))
Tags: ['OPEN', 'CLOSE']

Compiled grammar is:
#JSGF V1.0;
grammar default;
public <command> = (open { OPEN }|close { CLOSE }) the file;

Tagged rules are:
[PublicRule(name='command', expansion=Sequence(AlternativeSet(Literal('open'), Literal('close')), Literal('the file')))]

Tagged rules are:
[PublicRule(name='command', expansion=Sequence(AlternativeSet(Literal('open'), Literal('close')), Literal('the file')))]

Tags matching 'open the file' are: ['OPEN']
generated:: close the file


In [92]:
  
"""
Example showing use of some recursive expansion functions.
"""
from jsgf import *


def main():
    # Create a new public rule using speech alternatives.
#     rule = PublicRule("greet", Sequence(AlternativeSet("hello", "hey"), "there"))
    rule=rule1
    print(rule1)
    # Define a function to get the rule's current_match values using
    # flat_map_expansion.
    def get_values():
        return flat_map_expansion(rule.expansion, lambda x: x.current_match)

    # All values will initially be set to None.
    print("current_match values before matching: %s" % get_values())

    # Match a speech string and print the values again.
    rule.matches("hello there")
    print("current_match values after matching: %s" % get_values())

    # Use filter_expansion to get expansions with no current_match value.
    # This will get Literal("hey") which wasn't matched.
    print(filter_expansion(rule.expansion, lambda x: not x.current_match)[0])

    # Use map_expansion to print a representation of the rule's expansion tree
    def f(x):
        # Print the expansion with an indentation based on the number of
        # ancestors
        n, p = 0, x.parent
        while p:
            n += 4
            p = p.parent

        print("indention and ancestors%s%s" % (" " * n, x))

    map_expansion(rule.expansion, f)


if __name__ == '__main__':
    main()

PublicRule(name='music_play', expansion=Literal('[can you](put on| play)(<artist>|<song>)'))
current_match values before matching: [None]
current_match values after matching: [None]
Literal('[can you](put on| play)(<artist>|<song>)')
indention and ancestorsLiteral('[can you](put on| play)(<artist>|<song>)')


In [88]:
"""
Example use of the pyjsgf parse_grammar_string function.
The parse_grammar_file, parse_rule_string and parse_expansion_string functions
are also available and work in a similar way.
"""

from jsgf import parse_grammar_string

# Parse a grammar string with parse_grammar_string and get a Grammar object back.
grammar = parse_grammar_string(
    "#JSGF V1.0 UTF-8 en;"
    "grammar example;"
    "public <greet> = hello world {greeting};"
)

# Print it.
print(grammar)

# Get the rule that matches "hello world".
rule = grammar.find_matching_rules("hello world")[0]
print("Matching rule: %s" % rule)

# Tags are also parsed and will work as expected.
print("Matched tags: %s" % rule.matched_tags)

Grammar(version=1.0, charset=UTF-8, language=en, name=example)
Matching rule: Rule(name='greet', visible=True, expansion=Literal('hello world'))
Matched tags: ['greeting']


### Used for Our Grammar

In [141]:
!pip install dragonfly2

In [2]:
import dragonfly.parser as dparser

In [34]:
from dragonfly import ElementBase,Rule,Grammar,grammar
import dragonfly.grammar.elements as elements_

In [90]:
drag_grammar=Grammar('music_play')
drag_rule2=Rule(name='artist',visible=True,expansion='beatles |radio head |cake |pink floyd')
drag_rule3=Rule(name='song',visible=True,expansion='confortably numb |paranoid android |let it be |hey jude')
drag_grammar.add_rules(drag_rule2,drag_rule3)


### keyword spotting for multiple words

In [73]:
# !pip install sphinxwrapper

In [72]:
# from dragonfly import Grammar, CompoundRule, Dictation, get_engine
# engine = get_engine("sphinx")
# engine.config.START_ASLEEP = False
# class MyRule(CompoundRule):
#     spec = "hello <text>"
#     extras = [Dictation("text")]
#     def _process_recognition(self, node, extras):
#         # "world" will be printed in lowercase to be consistent with
#         # normal output from CMU Pocket Sphinx.
#         print(extras["text"])
# grammar = Grammar("dictation grammar")
# grammar.add_rule(MyRule())
# grammar.load()
# engine.mimic("hello WORLD")

In [52]:
a="majhe nav Vikas ahe"

In [53]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate


In [54]:
print(transliterate(a, sanscript.HK, sanscript.DEVANAGARI))

मझे नव् Vइकस् अहे




The Grammar

Let's cover the grammar very quickly: The Devanagari Block. As a developer, there are two character classes you'll want to concern yourself with:

Sign: This is a character that affects a previously-occurring character. Example, this character: ्. The light-colored circle indicates the location of the center of the character it is to be placed upon.
Letter / Vowel / Other: This is a character that may be affected by signs. Example, this character: क.
Combination result of ् and क: क्. But combinations can extend, so क् and षति will actually become क्षति (in this case, we right-rotate the first character by 90 degrees, modify some of the stylish elements, and attach it at the left side of the second character).

My answer here is not to solve the situation of these infinite (and tremendously beautiful) combinations, but simply clusters of singular letters and/or clusters of singular letters with their affecting, sign characters. If we are thinking "what are the characters of this Devanagari string?", then this is the right way to go, otherwise any combination of letters would form a unique character of a unique length, and then most of the concepts and algorithms associated with letter-systems would fail.

So, for instance, a symbol word would be...

(letter) (letter) (sign) (sign) (letter) (sign)
In this case, you'll want the result...

[
    0=>(letter),
    1=>(letter) (sign) (sign),
    2=>(letter) (sign),
]


